We'll now look at how we can 1) assemble a time series of imagery with stackstac and rioxarray and 2) calculate statistics of areas of interest within the time series. In order to accomplish this, we'll first need to define an area of interest that we will use to query and download our time series. 

We'll focus on farms in the Netherlands, the 2nd largest exporter of agricultural goods. Public Services on the Map (PDOK) provides [annual maps of cropland for Netherlands](https://service.pdok.nl/rvo/brpgewaspercelen/atom/v1_0/basisregistratie_gewaspercelen_brp.xml).

In [7]:
classes = gpd.read_file("../data/farms_amsterdam_2021-source-brp_pdok.shp")['category'].unique()

In [16]:
import geopandas as gpd

farms = gpd.read_file("../data/farms_amsterdam_2021-source-brp_pdok.shp")

Let's inspect our spatial data of farms.

In [18]:
farms

,id,category,gewas,gewascode,jaar,status,geometry
0,667.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((130081.907 495197.053 0.000, 13001..."
1,894.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((127163.634 495965.555 0.000, 12716..."
2,1547.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((128839.449 489607.228 0.000, 12883..."
3,1783.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((130436.562 495110.489 0.000, 13044..."
4,1784.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((130348.889 495569.052 0.000, 13033..."
...,...,...,...,...,...,...,...
3023,763681.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((126890.842 492959.327 0.000, 12689..."
3024,763682.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((126625.084 492891.611 0.000, 12662..."
3025,763683.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((126650.961 493224.888 0.000, 12665..."
3026,763684.0,Grasland,"Grasland, blijvend",265,2021,Voorgesteld,"POLYGON Z ((127373.919 494141.769 0.000, 12737..."


"Category" and "Status" look like important features. Let's assign these to variables for later use and inspect them.

In [19]:
classes = farms['category'].unique()
classes

array(['Grasland', 'Overige', 'Natuurterrein', 'Bouwland'], dtype=object)

These classes correspond to 

`'Grassland', 'Other', 'Nature reserve', 'Construction land'`

In [22]:
status = farms['status'].unique()
status

array(['Voorgesteld'], dtype=object)

'Voorgesteld' translates to "Proposed". Since this is a newly released dataset, the features haven't been fully manually reviewed and confirmed like prior years.

Let's search for and download Sentinel-2 scenes with stackstac that intersect this dataset. Since this dataset is for the year 2021, we will grab scenes during the summer of 2021. First, let's compute the bounding box of our area of interest, in lat,lon coordinates.

In [34]:
bbox = farms.to_crs(4326).total_bounds

In [35]:
bbox

array([ 4.87350746, 52.37275593,  5.08572375, 52.45899119])

In [36]:
from pystac_client import Client

api_url = "https://earth-search.aws.element84.com/v0"

client = Client.open(api_url)

# collection: Sentinel-2, Level 2A, COGs
collection = "sentinel-s2-l2a-cogs"

In [39]:
mysearch = client.search(
    collections=[collection],
    bbox=bbox,
    max_items=10,
    datetime="2021-06-01/2021-8-31",
    query=["eo:cloud_cover<5"]
)

In [40]:
print(mysearch.matched())

9


In [41]:
items = mysearch.get_all_items()

In [42]:
print(len(items))

9


In [43]:
for item in items:
    print(item)

<Item id=S2A_31UFU_20211221_0_L2A>
<Item id=S2B_31UFU_20211216_0_L2A>
<Item id=S2B_31UFU_20211024_0_L2A>
<Item id=S2B_31UFU_20211004_1_L2A>
<Item id=S2A_31UFU_20210909_0_L2A>
<Item id=S2B_31UFU_20210904_0_L2A>
<Item id=S2A_31UFU_20210823_0_L2A>
<Item id=S2B_31UFU_20210616_0_L2A>
<Item id=S2A_31UFU_20210601_0_L2A>


In [44]:
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2021-12-21 10:56:23+00:00
{'type': 'Polygon', 'coordinates': [[[6.071664488869862, 52.22257539160586], [4.464995307918359, 52.25346561204129], [4.498475093400055, 53.24019917467795], [6.1417542968794585, 53.20819279121764], [6.071664488869862, 52.22257539160586]]]}
{'datetime': '2021-12-21T10:56:23Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'gsd': 10, 'view:off_nadir': 0, 'proj:epsg': 32631, 'sentinel:utm_zone': 31, 'sentinel:latitude_band': 'U', 'sentinel:grid_square': 'FU', 'sentinel:sequence': '0', 'sentinel:product_id': 'S2A_MSIL2A_20211221T105451_N0301_R051_T31UFU_20211221T135048', 'sentinel:data_coverage': 100, 'eo:cloud_cover': 0, 'sentinel:valid_cloud_cover': True, 'created': '2021-12-21T16:58:37.019Z', 'updated': '2021-12-21T16:58:37.019Z'}


Let's save our search details in case we need to know what we queried later.

In [46]:
items.save_object("mysearch.json")

We'll use a subset of Sentinel-2's various bands to compute and compre spectral indices for our farms dataset.

In [47]:
assets = items[-1].assets  # last item's asset dictionary
print(assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [48]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

thumbnail: Thumbnail
overview: True color image
info: Original JSON metadata
metadata: Original XML metadata
visual: True color image
B01: Band 1 (coastal)
B02: Band 2 (blue)
B03: Band 3 (green)
B04: Band 4 (red)
B05: Band 5
B06: Band 6
B07: Band 7
B08: Band 8 (nir)
B8A: Band 8A
B09: Band 9
B11: Band 11 (swir16)
B12: Band 12 (swir22)
AOT: Aerosol Optical Thickness (AOT)
WVP: Water Vapour (WVP)
SCL: Scene Classification Map (SCL)


In [49]:
print(assets["thumbnail"].href)

https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/FU/2021/6/1/0/preview.jpg


A Sentinel-2 image is quite large. Instead of using `.rio.clip_box()` as in the last epsidoe, we can read data using rasterio's `Window` class to only grab the data we need. Befor edoing so, it helps to check the block size fo the raster...

NameError: name 'red_href' is not defined

And to define the window area we want to read in terms of pixel coordinates. We can use rasterio/rioxarray's transform functionality to convert our `bbox` from geographic coordinates to utm coordinates (the projection of our rasters) and finally convert to pixel coordinates (the unit necessary for windowed reads).

In [64]:
transform

Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 5900040.0)

In [69]:
farms.crs

<Projected CRS: EPSG:28992>
Name: Amersfoort / RD New
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Netherlands - onshore, including Waddenzee, Dutch Wadden Islands and 12-mile offshore coastal zone.
- bounds: (3.2, 50.75, 7.22, 53.7)
Coordinate Operation:
- name: RD New
- method: Oblique Stereographic
Datum: Amersfoort
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [71]:
bbox_utm = farms.to_crs(32631).total_bounds

In [80]:
rasterio.open(red_href).overviews(1)

[2, 4, 8, 16]

In [73]:
from rasterio.windows import Window
import rasterio
red_href = assets["B04"].href
nir_href = assets["B08"].href
transform = rioxarray.open_rasterio(red_href).rio.transform()
window_of_interest = rasterio.windows.from_bounds(*bbox_utm, transform=transform)

In [83]:
window_of_interest

Window(col_off=2730.5694674572223, row_off=8629.521067048307, width=1448.0200055093737, height=934.8902779202908)

# TODO having some problems here. this reads the full size image for some reason rather than the window.

looking to accomplish this, but with rioxarray: https://gis.stackexchange.com/questions/336874/get-a-window-from-a-raster-in-rasterio-using-coordinates-instead-of-row-column-o

In [76]:
rioxarray.open_rasterio(red_href, window=window_of_interest, windowed=True)

<xarray.DataArray (band: 1, y: 10980, x: 10980)>
[120560400 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 5.9e+06 5.9e+06 5.9e+06 ... 5.79e+06 5.79e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0